# Beam Search Example
This example demonstrates how to use the `BeamSearch` search algorithm with a simple reasoning task. The Beam Search algorithm is a heuristic search that explores a graph by expanding the most promising nodes in a limited set. Search is performed in a breadth-first manner, keeping track of the best candidates at each level. The algorithm continues until a stopping criterion is met, such as reaching a maximum number of steps or finding a satisfactory solution.

**Imports**

This example will use LMStudio for local model serving and Google Gemini for generation via external API. CoT-Forge also supports major LLM providers like OpenAI, Groq, and Anthropic. Download LMStudio [here](https://lmstudio.ai/), download a model and run a local server.

In [20]:
from cot_forge import BeamSearch, CoTBuilder, LLMJudgeVerifier, ProbabilityFinalAnswerScorer
from cot_forge.llm import GeminiProvider, LMStudioProvider

### Problem
The following is a problem from the [The Verbal Reasoning Challenge dataset by Northeastern University Programming Research Lab](https://huggingface.co/datasets/nuprl/reasoning-weekly). It is based on the "off-air challenges" from the NPR Sunday Puzzle Challenge, which are designed to be understandable by any adult in the United States.

In [ ]:
question = r"""Name two parts of the human body. Put them together one after the other. Change the seventh letter in the result to the next letter of the alphabet to name something that's often found in books. What is it?"""
ground_truth = "Footnote"

### Using CoTBuilder class to run beam search

In [4]:
# We'l use meta-llama-3-8b-instruct as our search_llm.
# Gemini will be used for the verifier and scorer.
llama = LMStudioProvider(model_name="meta-llama-3-8b-instruct")
gemini = GeminiProvider()

**Beam search parameters:**
  * max_depth - The maximum number of steps any beam can take before stopping.
  * beam_width - The number of beams to keep at each step.
  * branching_factor - Number of branches to consider from each active beam (i.e., if we have 2 active beams, and branching_factor=3, we will consider 6 branches, keeping the two best).

In [5]:
builder = CoTBuilder(
    search_llm=llama, # Llama will do the reasoning in the search 
    post_processing_llm=gemini, # Gemini will convert reasoning into natural language
    search=BeamSearch(max_depth=3, beam_width=3, branching_factor=2),
    verifier=LLMJudgeVerifier(gemini, strict=False), # Gemini will verify final answers. Strict=False means it will not require exact match.
    scorer=ProbabilityFinalAnswerScorer(gemini) # Gemini will score branch options based on likelihood of the reaching the final answer.
)

CoTBuilder.process() returns a tuple of a [SearchResult](../src/cot_forge/reasoning/types.py#L137) object and a dictionary with the natural language reasoning text.

In [ ]:
search_result, reasoning = builder.process(
  question=question,
  llm_kwargs={"temperature": 0.0},
  ground_truth_answer=ground_truth, 
  only_successful=True # Only successful branches will be processed to natural language and included in `reasoning` output.
)

### Examining the search result

In [8]:
search_result

SearchResult(question=Name two parts of the human bo..., ground_truth_answer=Footnote, num_terminal_nodes=3, success=True, successful_nodes=1, metadata={'depth': 3, 'reason': 'Max depth reached'})

Examining our search result, we see that of the three terminal nodes (beams), one is successful. Let's examine it further.

In [14]:
successful_beam = search_result.get_successful_terminal_nodes()[0]
for i, node in enumerate(successful_beam.get_full_node_chain()):
  print(f"Step {i}: {node}")

Step 0: ReasoningNode(strategy=initialize, success=False, final=False, cot_steps=5)
Step 1: ReasoningNode(strategy=validation, success=True, final=True, cot_steps=5)


We see that this beam got to the correct answer in two steps with the following strategies: initialization -> validation.

Each reasoning node has a cot property which is a list of reasoning steps of types 'review', 'inner thinking' or 'final conclusion'. By printing out the whole reasoning chain, we can see how the model arrived at the final answer.

In [15]:
from pprint import pprint

pprint(successful_beam.get_full_cot())

[{'action': 'Review',
  'content': 'The question asks to name two parts of the human body, combine '
             'them, change the seventh letter in the result, and identify '
             'something often found in books.'},
 {'action': 'Inner Thinking',
  'content': "Two parts of the human body are 'hand' and 'foot'. I will put "
             "them together to get 'handfoot'.",
  'title': 'Step 1: Name Two Parts of the Human Body'},
 {'action': 'Inner Thinking',
  'content': "The seventh letter in 'handfoot' is 'n'. If I change it to the "
             "next letter of the alphabet, which is 'o', I get 'handfooT'.",
  'title': 'Step 2: Change the Seventh Letter in the Result'},
 {'action': 'Inner Thinking',
  'content': 'A table of contents (TOC) is often found in books. It starts '
             "with the word 'Table' and has a similar sound to 'tableof', "
             "which can be derived from 'handfooT'. Therefore, I conclude that "
             'a TOC is something often found in 

### Natural language reasoning
Finally, let's examine the natural language reasoning text. This is a human-readable version of the reasoning steps taken by the model, which converts the structured reasoning chain above into a more human format. This text can be used for fine-tuning reasoning models.

*Note:* Because we specified `only_successful=True` in builder.process(), we only will have converted the one successful beam to natural language. If we had set `only_successful=False`, we would have converted all beams to natural language, including the unsuccessful ones.

In [17]:
# Reasoning is a dictionary object. We're most interested in 'chain_of_thought_responses', which contains a list of all of the natural language reasoning generations.
print(reasoning['chain_of_thought_responses'][0])

<thinking>Okay, two parts of the human body... let's see. How about 'hand' and 'foot'?

Putting them together gives me 'handfoot'.

Hmm, the seventh letter... that's the 't' in 'handfoot'. Wait, there isn't a seventh letter! Let's try something else.

Okay, think, think... what about 'head' and 'heart'? That sounds promising.

So, 'headheart'... Now the seventh letter *is* there, and it's a 't'.

Changing 't' to the next letter in the alphabet, 'u', makes it 'headhearU'. That doesn't look right.

Actually, let's try 'head' and 'ear'. That gives us 'headear'.

Seventh letter... oh wait, still no seventh letter. This is trickier than I thought.

Okay, back to the drawing board. How about 'fore' and 'arm'?

That makes 'forearm'. Ah-ha! The seventh letter is 'm'.

Change 'm' to 'n' and we get 'forearn'. Still doesn't seem right.

Maybe I'm focusing too much on just following the steps directly. Maybe I should think about what's often found in books first.

What about an author? No, that do